# Week 2: Baseline Modeling with Logistic Regression

## Objectives
1. Load the processed dataset from Week 1
2. Split data into train/test sets with stratification
3. Scale features appropriately
4. Train a dummy baseline classifier
5. Train logistic regression with class balancing
6. Evaluate models comprehensively
7. Interpret model coefficients
8. Save model and results

## Timeline
**Week 2 (Nov 25 - Dec 1)** - Due: Dec 1

---

## Setup

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import joblib

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report,
    roc_curve, roc_auc_score,
    precision_recall_curve, average_precision_score
)

# Settings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
pd.set_option('display.max_columns', None)

# Random seed
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print("✅ Libraries imported successfully")

In [ ]:
# Set up paths
project_root = Path.cwd().parent
data_dir = project_root / 'data'
processed_data_dir = data_dir / 'processed'
models_dir = project_root / 'models'
figures_dir = project_root / 'figures'

# Create directories
models_dir.mkdir(parents=True, exist_ok=True)

print(f"Project root: {project_root}")
print(f"Models directory: {models_dir}")

## 1. Load Processed Dataset

In [ ]:
# Load the dataset created in Week 1
data_file = processed_data_dir / 'hits_dataset.csv'

if not data_file.exists():
    raise FileNotFoundError(
        f"❌ Dataset not found at {data_file}\n"
        "Please run notebook 01_Week1_Data_Setup_EDA.ipynb first!"
    )

df = pd.read_csv(data_file)
print(f"✅ Loaded dataset: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nClass distribution:")
print(df['is_hit'].value_counts())
print(f"\nFirst few rows:")
df.head()

## 2. Prepare Features and Target

In [ ]:
# Identify feature columns (numeric columns except target and metadata)
# Exclude: is_hit, year, track names, artist names
exclude_cols = ['is_hit', 'year']

# Also exclude any text columns
text_cols = df.select_dtypes(include=['object']).columns.tolist()
exclude_cols.extend(text_cols)

# Get feature columns
feature_cols = [col for col in df.columns if col not in exclude_cols]

print(f"Feature columns ({len(feature_cols)}):")
print(feature_cols)

# Prepare X and y
X = df[feature_cols].values
y = df['is_hit'].values

print(f"\nX shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"\nTarget distribution:")
unique, counts = np.unique(y, return_counts=True)
for label, count in zip(unique, counts):
    print(f"  Class {label}: {count:,} ({count/len(y)*100:.2f}%)")

## 3. Train/Test Split

**Important:** We use stratified split to maintain class balance in both sets.

In [ ]:
# Stratified train/test split
TEST_SIZE = 0.2

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=TEST_SIZE, 
    random_state=RANDOM_SEED,
    stratify=y  # Maintain class distribution
)

print(f"Train set size: {len(X_train):,} ({(1-TEST_SIZE)*100:.0f}%)")
print(f"Test set size:  {len(X_test):,} ({TEST_SIZE*100:.0f}%)")

print(f"\nTrain set class distribution:")
unique, counts = np.unique(y_train, return_counts=True)
for label, count in zip(unique, counts):
    print(f"  Class {label}: {count:,} ({count/len(y_train)*100:.2f}%)")

print(f"\nTest set class distribution:")
unique, counts = np.unique(y_test, return_counts=True)
for label, count in zip(unique, counts):
    print(f"  Class {label}: {count:,} ({count/len(y_test)*100:.2f}%)")

## 4. Feature Scaling

Logistic regression benefits from scaled features (mean=0, std=1).

In [ ]:
# Fit scaler on training data only (avoid data leakage)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✅ Features scaled")
print(f"\nTraining set statistics (after scaling):")
print(f"  Mean: {X_train_scaled.mean():.6f} (should be ~0)")
print(f"  Std:  {X_train_scaled.std():.6f} (should be ~1)")

# Save scaler for future use
joblib.dump(scaler, models_dir / 'scaler.pkl')
print(f"\n✅ Saved scaler to {models_dir / 'scaler.pkl'}")

## 5. Baseline Model: Dummy Classifier

Before building a real model, let's establish a baseline using a simple strategy.

In [ ]:
# Dummy classifier (predicts most frequent class)
dummy_clf = DummyClassifier(strategy='most_frequent', random_state=RANDOM_SEED)
dummy_clf.fit(X_train_scaled, y_train)
y_pred_dummy = dummy_clf.predict(X_test_scaled)

print("DUMMY CLASSIFIER RESULTS (Baseline)")
print("="*60)
print(f"Strategy: Always predict most frequent class")
print(f"\nAccuracy:  {accuracy_score(y_test, y_pred_dummy):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_dummy, zero_division=0):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred_dummy, zero_division=0):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred_dummy, zero_division=0):.4f}")

print(f"\n⚠️  This is our baseline to beat!")
print(f"    A good model should perform significantly better than this.")

## 6. Logistic Regression with Class Balancing

**Key Strategy:** Use `class_weight='balanced'` to handle class imbalance.

This automatically adjusts weights to be inversely proportional to class frequencies.

In [ ]:
# Train logistic regression
print("Training Logistic Regression...\n")

logreg = LogisticRegression(
    class_weight='balanced',  # Handle class imbalance
    random_state=RANDOM_SEED,
    max_iter=1000,
    solver='lbfgs'
)

logreg.fit(X_train_scaled, y_train)

print("✅ Model trained successfully")

# Make predictions
y_pred = logreg.predict(X_test_scaled)
y_pred_proba = logreg.predict_proba(X_test_scaled)[:, 1]

print("✅ Predictions generated")

## 7. Model Evaluation

### 7.1 Basic Metrics

In [ ]:
# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
pr_auc = average_precision_score(y_test, y_pred_proba)

print("LOGISTIC REGRESSION RESULTS")
print("="*60)
print(f"Accuracy:  {accuracy:.4f}  ← Overall correctness")
print(f"Precision: {precision:.4f}  ← Of predicted hits, how many are correct?")
print(f"Recall:    {recall:.4f}  ← Of actual hits, how many did we catch?")
print(f"F1 Score:  {f1:.4f}  ← Harmonic mean of precision and recall")
print(f"ROC-AUC:   {roc_auc:.4f}  ← Overall discrimination ability")
print(f"PR-AUC:    {pr_auc:.4f}  ← Precision-recall trade-off (better for imbalanced data)")

print(f"\n⚠️  For imbalanced data, focus on:")
print(f"   • Recall (catch as many hits as possible)")
print(f"   • F1 Score (balance precision and recall)")
print(f"   • PR-AUC (more informative than ROC-AUC for imbalanced data)")

### 7.2 Confusion Matrix

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)

fig, ax = plt.subplots(figsize=(8, 6))

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Non-Hit', 'Hit'],
            yticklabels=['Non-Hit', 'Hit'],
            cbar_kws={'label': 'Count'},
            ax=ax)

ax.set_xlabel('Predicted Label', fontsize=12)
ax.set_ylabel('True Label', fontsize=12)
ax.set_title('Confusion Matrix - Logistic Regression', fontsize=14, fontweight='bold')

# Add percentage annotations
for i in range(2):
    for j in range(2):
        percentage = cm[i, j] / cm[i].sum() * 100
        ax.text(j + 0.5, i + 0.7, f'({percentage:.1f}%)', 
                ha='center', va='center', fontsize=10, color='gray')

plt.tight_layout()
plt.savefig(figures_dir / 'logreg_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: logreg_confusion_matrix.png")

# Print detailed breakdown
print(f"\nConfusion Matrix Breakdown:")
print(f"  True Negatives  (TN): {cm[0, 0]:,}  ← Correctly predicted non-hits")
print(f"  False Positives (FP): {cm[0, 1]:,}  ← Non-hits wrongly predicted as hits")
print(f"  False Negatives (FN): {cm[1, 0]:,}  ← Hits wrongly predicted as non-hits")
print(f"  True Positives  (TP): {cm[1, 1]:,}  ← Correctly predicted hits")

### 7.3 Classification Report

In [ ]:
# Detailed classification report
print("Classification Report:")
print("="*60)
print(classification_report(y_test, y_pred, 
                          target_names=['Non-Hit', 'Hit'],
                          digits=4))

### 7.4 ROC Curve

In [ ]:
# ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

fig, ax = plt.subplots(figsize=(8, 6))

ax.plot(fpr, tpr, linewidth=2, label=f'Logistic Regression (AUC = {roc_auc:.4f})')
ax.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier (AUC = 0.5000)')

ax.set_xlabel('False Positive Rate', fontsize=12)
ax.set_ylabel('True Positive Rate', fontsize=12)
ax.set_title('ROC Curve - Logistic Regression', fontsize=14, fontweight='bold')
ax.legend(loc='lower right', fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(figures_dir / 'logreg_roc_curve.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: logreg_roc_curve.png")

### 7.5 Precision-Recall Curve

**More informative than ROC for imbalanced datasets!**

In [ ]:
# Precision-Recall curve
precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_pred_proba)

fig, ax = plt.subplots(figsize=(8, 6))

ax.plot(recall_curve, precision_curve, linewidth=2, 
        label=f'Logistic Regression (AP = {pr_auc:.4f})')

# Baseline (random classifier for imbalanced data)
baseline = (y_test == 1).sum() / len(y_test)
ax.axhline(y=baseline, color='k', linestyle='--', linewidth=1,
          label=f'Random Classifier (AP = {baseline:.4f})')

ax.set_xlabel('Recall', fontsize=12)
ax.set_ylabel('Precision', fontsize=12)
ax.set_title('Precision-Recall Curve - Logistic Regression', fontsize=14, fontweight='bold')
ax.legend(loc='upper right', fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])

plt.tight_layout()
plt.savefig(figures_dir / 'logreg_pr_curve.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: logreg_pr_curve.png")

## 8. Feature Importance (Coefficients)

Logistic regression coefficients tell us which features are most important for predicting hits.

In [ ]:
# Get coefficients
coefficients = pd.DataFrame({
    'Feature': feature_cols,
    'Coefficient': logreg.coef_[0]
})
coefficients['Abs_Coefficient'] = np.abs(coefficients['Coefficient'])
coefficients = coefficients.sort_values('Abs_Coefficient', ascending=False)

print("Feature Coefficients (sorted by absolute value):")
print("="*60)
for idx, row in coefficients.iterrows():
    direction = "increases" if row['Coefficient'] > 0 else "decreases"
    print(f"{row['Feature']:20s} {row['Coefficient']:+.4f}  ← {direction} hit probability")

print(f"\nInterpretation:")
print(f"  • Positive coefficient: Higher feature value → More likely to be a HIT")
print(f"  • Negative coefficient: Higher feature value → More likely to be a NON-HIT")
print(f"  • Larger absolute value: Stronger effect")

In [ ]:
# Visualize coefficients
fig, ax = plt.subplots(figsize=(10, 6))

colors = ['#e74c3c' if c < 0 else '#2ecc71' for c in coefficients['Coefficient']]
bars = ax.barh(range(len(coefficients)), coefficients['Coefficient'], color=colors)

ax.set_yticks(range(len(coefficients)))
ax.set_yticklabels(coefficients['Feature'])
ax.set_xlabel('Coefficient Value', fontsize=12)
ax.set_title('Feature Coefficients - Logistic Regression', fontsize=14, fontweight='bold')
ax.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
ax.grid(True, alpha=0.3, axis='x')

# Add legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#2ecc71', label='Increases hit probability'),
    Patch(facecolor='#e74c3c', label='Decreases hit probability')
]
ax.legend(handles=legend_elements, loc='lower right')

plt.tight_layout()
plt.savefig(figures_dir / 'logreg_coefficients.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: logreg_coefficients.png")

## 9. Save Model

In [ ]:
# Save the trained model
model_file = models_dir / 'baseline_logreg.pkl'
joblib.dump(logreg, model_file)

print(f"✅ Saved model to: {model_file}")

# Save metrics for comparison later
metrics = {
    'model': 'Logistic Regression',
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1_score': f1,
    'roc_auc': roc_auc,
    'pr_auc': pr_auc
}

metrics_df = pd.DataFrame([metrics])
metrics_df.to_csv(models_dir / 'baseline_metrics.csv', index=False)
print(f"✅ Saved metrics to: {models_dir / 'baseline_metrics.csv'}")

## 10. Summary Report

In [ ]:
print("="*80)
print("WEEK 2 SUMMARY REPORT")
print("="*80)

print(f"\n1. MODEL PERFORMANCE")
print(f"\n   Dummy Baseline (Most Frequent):")
print(f"     Accuracy: {accuracy_score(y_test, y_pred_dummy):.4f}")
print(f"     F1 Score: {f1_score(y_test, y_pred_dummy, zero_division=0):.4f}")

print(f"\n   Logistic Regression (Balanced):")
print(f"     Accuracy:  {accuracy:.4f}")
print(f"     Precision: {precision:.4f}")
print(f"     Recall:    {recall:.4f}")
print(f"     F1 Score:  {f1:.4f}")
print(f"     ROC-AUC:   {roc_auc:.4f}")
print(f"     PR-AUC:    {pr_auc:.4f}")

print(f"\n2. KEY INSIGHTS")
print(f"\n   Top 3 Positive Predictors (increase hit probability):")
top_positive = coefficients[coefficients['Coefficient'] > 0].head(3)
for i, (idx, row) in enumerate(top_positive.iterrows(), 1):
    print(f"     {i}. {row['Feature']}: {row['Coefficient']:+.4f}")

print(f"\n   Top 3 Negative Predictors (decrease hit probability):")
top_negative = coefficients[coefficients['Coefficient'] < 0].head(3)
for i, (idx, row) in enumerate(top_negative.iterrows(), 1):
    print(f"     {i}. {row['Feature']}: {row['Coefficient']:+.4f}")

print(f"\n3. MODEL STRENGTHS & WEAKNESSES")
if recall > 0.6:
    print(f"   ✅ Good recall: Catching {recall*100:.1f}% of actual hits")
else:
    print(f"   ⚠️  Low recall: Only catching {recall*100:.1f}% of actual hits")

if precision > 0.5:
    print(f"   ✅ Decent precision: {precision*100:.1f}% of predicted hits are correct")
else:
    print(f"   ⚠️  Low precision: Only {precision*100:.1f}% of predicted hits are correct")

print(f"\n4. NEXT STEPS (Week 3)")
print(f"   → Train XGBoost model (handle non-linear patterns)")
print(f"   → Hyperparameter tuning")
print(f"   → SHAP analysis for deeper interpretation")
print(f"   → Compare XGBoost vs Logistic Regression")

print(f"\n5. FILES CREATED")
print(f"   → {model_file}")
print(f"   → {models_dir / 'scaler.pkl'}")
print(f"   → {figures_dir / 'logreg_confusion_matrix.png'}")
print(f"   → {figures_dir / 'logreg_roc_curve.png'}")
print(f"   → {figures_dir / 'logreg_pr_curve.png'}")
print(f"   → {figures_dir / 'logreg_coefficients.png'}")

print("\n" + "="*80)
print("✅ WEEK 2 COMPLETE - Baseline model established!")
print("="*80)

---

## ✅ Week 2 Deliverables Complete!

You now have:
1. ✅ Trained logistic regression baseline
2. ✅ Comprehensive evaluation metrics
3. ✅ Feature importance analysis
4. ✅ Multiple visualizations
5. ✅ Saved model for future comparison

### Key Takeaways:
- Logistic regression provides an **interpretable** baseline
- Class balancing helps handle imbalanced data
- For imbalanced data, **F1, Recall, and PR-AUC** are more important than accuracy
- Coefficients reveal which audio features matter most for hits

### Next Steps:
Proceed to **03_Week3_XGBoost_SHAP.ipynb** for advanced modeling with XGBoost and SHAP interpretation.

---